## The individual recomendations procedure:
    - Ask user to rate 10 movies
    - Get predictions for a user using ALS lenskit algorithm    
    - make a list of some movies with highest predicted scores
    - rerank based on cosine similarity

In [1]:
import pandas as pd
import numpy as np


In [2]:
#preoare item vectors for each movie
import numpy as np
from numpy import dot
from numpy.linalg import norm

#tag_genome_matrix = genome_df.pivot(index='movieId', columns='tagId', values='relevance')
tag_rel_matrix_df = pd.read_csv('Preprocessed_Data/post_processed_movie_tag.csv')
tag_rel_matrix_df=  tag_rel_matrix_df.rename(columns={'item_id':'item'})

In [3]:
movies_df = pd.read_csv('DataSets_/movie.csv')
movies_df = movies_df.rename(columns={'movieId': 'item', 'title' : 'title_original'})
movies_df.head(10)

,item,title_original,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [4]:
ratings_df = pd.read_csv('Preprocessed_Data/Latest_100000_ratings.csv', index_col = 0)
ratings_df = ratings_df.rename(columns={'movieId': 'item', 'userId' : 'user'})
ratings_df.head(10)

,user,item,rating,timestamp
12675921,87586,7151,3.5,2015-03-31 06:40:02
12898527,89081,52458,4.0,2015-03-31 06:11:28
12898546,89081,55232,3.5,2015-03-31 06:11:26
2508834,16978,2093,3.5,2015-03-31 06:03:17
7819902,53930,118706,3.5,2015-03-31 06:00:51
2509323,16978,106642,3.0,2015-03-31 06:00:28
10151906,70232,58998,2.5,2015-03-31 05:55:28
8378411,57814,1590,3.5,2015-03-31 05:51:49
8378415,57814,2311,4.5,2015-03-31 05:51:23
8378501,57814,94864,3.5,2015-03-31 05:50:52


In [5]:
# keep only items that are in the ratings_df and in the tag_rel_matrix_df

df1= tag_rel_matrix_df
df2= ratings_df
# Identify the common item_ids
common_item_ids = df1[df1['item'].isin(df2['item'])]['item'].unique()

# Filter both dataframes to only include these common item_ids
df1 = df1[df1['item'].isin(common_item_ids)]
df2 = df2[df2['item'].isin(common_item_ids)]

assert len(df1[df1['item'].isin(df2['item'])]['item'].unique()) == len(df2[df2['item'].isin(df1['item'])]['item'].unique())


In [6]:
# given user_id make user_tag_profile_vector
def make_user_tag_profile(user_id):
    # tag_rel_matrix_df = tag_rel_matrix_df[1:]
    user_tag_profile_vector = np.zeros(len(tag_rel_matrix_df.columns[1:]))
    user_ratings_df = ratings_df[ratings_df['user']==user_id]
    summ = 0
    for index in user_ratings_df.index:
      user_rating =  user_ratings_df.loc[index]
      vector = np.array(tag_rel_matrix_df[tag_rel_matrix_df['item'] == user_rating['item']])
      vector = vector[0, 1:]
      rating = user_rating['rating']
      user_tag_profile_vector += vector * rating
      summ += user_rating['rating']
    user_tag_profile_vector /= summ
    return user_tag_profile_vector


In [10]:
from lenskit.algorithms import Recommender, als, item_knn, user_knn


# We use the collaborative filtering algorithm ALS.BiasedMF, that use cosine similarity
num_recs = 10  # Number of recommendations to generate
als = als.BiasedMF(50)
recsys = Recommender.adapt(als)
recsys.fit(ratings_df)

AttributeError: module 'scipy' has no attribute '__version__'

In [ ]:
def return_movie_title_from_id(item_id):
  return movies_df[movies_df['item']==item_id]['title_original'].iloc[0]

In [ ]:
def cos_sim_recommended_movies(user_tag_profile, recommended_movies_df, matrix_df):
    for index, row in recommended_movies_df.iterrows():
        # Extract item_tag_vector
        item_tag_vector = matrix_df[matrix_df['item'] == row['item']].drop(columns=['item']).values

        # If item_tag_vector is not empty
        if item_tag_vector.size > 0:
            # Ensure item_tag_vector is 1-D for dot product
            item_tag_vector = item_tag_vector[0]

            # Compute cosine similarity
            cos_sim = dot(user_tag_profile, item_tag_vector) / (norm(user_tag_profile) * norm(item_tag_vector))
            new_score = row['score'] * (cos_sim + 0.5)
            recommended_movies_df.loc[index, 'new_score'] = new_score
        else:
            # Handle the case where item_tag_vector is empty (can adjust as needed)
            recommended_movies_df.loc[index, 'new_score'] = 0

    return recommended_movies_df.sort_values(by=['new_score'], ascending=False)

In [ ]:
# mehod to get recommendations for a user with Lenskit ItemItem + user_tag_profile
def individual_recommender_pipeline(user_id):

    # get user_tag_profile_vector
    user_tag_profile_vector = make_user_tag_profile(user_id)

    # get 10 recommendations
    selected_movies_useruser = recsys.recommend(user_id, 10)

    # compute cosine similarity and new scores
    result = cos_sim_recommended_movies(user_tag_profile_vector, selected_movies_useruser,tag_rel_matrix_df)

    result['movie_title'] = result['item'].apply(return_movie_title_from_id)
    return result # reranked reccomendations

In [ ]:
def calc_new_score(score, user, item,matrix_df):
    user_tag_profile = make_user_tag_profile(user)
    item_tag_vector = matrix_df[matrix_df['item'] == item].drop(columns=['item']).values
    item_tag_vector = item_tag_vector[0]
    cos_sim = dot(user_tag_profile, item_tag_vector) / (norm(user_tag_profile) * norm(item_tag_vector))
    return (cos_sim + 0.5) * score

def mathias_algo(user, item,algo = als):
    itemArr = [item]
    score = algo.predict_for_user(user, itemArr)
    return calc_new_score(score,user,item,tag_rel_matrix_df)

In [ ]:
mathias_algo(87586,318)

In [ ]:
def list_n_tags_recommended_movies(movie_ids, n):
  sum_rows = tag_rel_matrix_df[tag_rel_matrix_df['item'].isin(movie_ids)].sum(axis=0)[1:]
  tags = tag_rel_matrix_df.columns[1:].tolist()
  sorted_lists = sorted(zip(sum_rows, tags), reverse=True)
  sorted_values = [x for _, x in sorted_lists]
  return sorted_values[1:n+1]

def list_n_tags_user(user_id, n):
  user_tag_profile = make_user_tag_profile(user_id)
  tags = tag_rel_matrix_df.columns[1:].tolist()
  sorted_lists = sorted(zip(user_tag_profile, tags), reverse=True)
  sorted_values = [x for _, x in sorted_lists]
  return sorted_values[1:n+1]

In [ ]:
def individual_explain(user_id, n):
  user_tags = list_n_tags_user(user_id, n)
  items = individual_recommender_pipeline(user_id)
  item_tags = list_n_tags_recommended_movies(items, n)

  # rec_item0 = items.iloc[0, items.columns.get_loc('movie_title')]
  # rec_item0 = items.iloc[0, items.columns.get_loc('movie_title')]
  # rec_item0 = items.iloc[0, items.columns.get_loc('movie_title')]
  # rec_item0 = items.iloc[0, items.columns.get_loc('movie_title')]
  rec_item = items.iloc[0, items.columns.get_loc('movie_title')]

  return "Recommended item" + ": [" + rec_item +"] Because you like movies about: " + ", ".join(user_tags) + ", we recommend movies with the following tags: " + ", ".join(item_tags)
